# Preliminaries

In this section, we briefly introduce the basic concepts of graph representation as well as the general vertex and edge updating mechanism of Graph Neural Networks (GNNs).

## Graph Representation

A graph $\mathcal{G} = (\mathcal{V}, \mathcal{E})$ is made up of a set of vertices $\mathcal{V} \subseteq \{\mathbf{v}_i \in \mathbb{R}^{1 \times K} \}$, and edges $\mathcal{E} \subseteq \{ \mathbf{e}_{i,j} = \mathbf{e}(\mathbf{v}_i, \mathbf{v}_j) \mid \mathbf{v}_i, \mathbf{v}_j \in \mathcal{V},  i \neq j \}$, where $\mathbf{v}_i$ represents $K$ attributes of the $i_{th}$ object/component in the pre-defined graph or non-graph data sample and $\mathbf{e}_{i,j}$ represents the edge feature that defines the relationship between the vertices $\mathbf{v}_i$ and $\mathbf{v}_j$. Each pair of vertices can be only connected by at most one undirected edge or two directed edges. A standard way to describe such edges is through the adjacency matrix $\mathcal{A} \in \mathbb{R}^{|\mathcal{V}| \times |\mathcal{V}|}$, where all vertices in a graph are ordered so that each vertex indexes a specific row and column. As a result, the presence of each edge $\mathbf{e}_{i,j}$ can be described by a binary value $\mathcal{A}_{i,j} = 1$ if $\mathbf{v}_i$ and $\mathbf{v}_j$ are connected or $\mathcal{A}_{i,j} = 0$ otherwise. Specifically, the adjacent matrix is always symmetric if all edges are undirected, but can be non-symmetric if one or more directed edges exist. Instead of using a binary value, some studies \cite{dwivedi2020benchmarking,song2021uncertain,isufi2021edgenets} also build adjacency matrices with continuous real values to describe the strength of association between each pair of vertices.

## Vertex/Edge Updating of Message-Passing GNNs

Recently, message-passing Graph Neural Networks (GNNs) \cite{kipf2016semi,xu2018powerful,bresson2017residual,velivckovic2017graph}, including Graph Convolution Networks (GCNs), become dominant models for a wide variety of graph analysis tasks. Given a GNN $G$, its $l_{th}$ layer $G^l$ takes the graph $\mathcal{G}^{l-1} = (\mathcal{V}^{l-1}, \mathcal{E}^{l-1})$ produced by the ${l-1}_{th}$ layer $G^{l-1}$, as the input, and generates a new graph $\mathcal{G}^{l} = (\mathcal{V}^l, \mathcal{E}^l)$, which can be formulated as:
\begin{equation}
    \mathcal{G}^{l} = G^l(\mathcal{G}^{l-1})
\end{equation}
Specifically, the vertex feature $\mathbf{v}^l_i$ in $\mathcal{G}^{l}$ is computed based on: (i) its previous status $\mathbf{v}_i^{l-1}$ in $\mathcal{G}^{l-1}$; (ii) a set of adjacent vertices of the $\mathbf{v}_i^{l-1} \in \mathcal{G}^{l-1}$ (denoted as the $\mathbf{v}_j^{l-1} \subseteq \mathcal{N}(\mathbf{v}_i^{l-1})$, where $\mathcal{A}^{l-1}_{i,j} = 1$, and $\mathcal{A}^{l-1}$ is the adjacent matrix of the $\mathcal{G}^{l-1}$); and (iii) a set of edge features $\mathbf{e}_{j,i}^{l-1}$ that represent the relationship between every $\mathbf{v}_j^{l-1}$ and $\mathbf{v}_i^{l-1}$ in $\mathcal{N}(\mathbf{v}_i^{l-1})$. Here, the message $ \mathbf{m}_{\mathcal{N}(\mathbf{v}^{l-1}_i)}$ is produced by aggregating all adjacent vertices of $\mathbf{v}_i^{l-1}$ through related edges $\mathbf{e}_{j,i}^{l-1}$, which can be formulated as:
\begin{equation}
\begin{split}
& \mathbf{m}_{\mathcal{N}(\mathbf{v}^{l-1}_i)} = M(\mathbin\Vert ^{N}_{j=1} f(\mathbf{v}_j^{l-1},\mathbf{e}_{j,i}^{l-1})) \\
& f(\mathbf{v}_j^{l-1},\mathbf{e}_{j,i}^{l-1}) = 0 \quad \text{subject to} \quad \mathcal{A}^{l-1}_{i,j} = 0
\end{split}
\label{eq:message-passing}
\end{equation}
where $M$ is a differentiable function that aggregates messages produced from all adjacent vertices; $N$ denotes the number of vertices in the graph $\mathcal{G}^{l-1}$; $f(\mathbf{v}_j^{l-1},\mathbf{e}_{j,i}^{l-1})$ is a differentiable function defining the influence of an adjacent vertex $\mathbf{v}_j^{l-1}$ on the vertex $\mathbf{v}_i^{l-1}$ through their edge $\mathbf{e}_{j,i}^{l-1}$; and $\mathbin\Vert $ is the aggregation operator to combine messages of all adjacent vertices of the $\mathbf{v}_i^{l-1}$. As a result, the vertex feature $\mathbf{v}^l_i$ can be updated as:
\begin{equation}
\begin{split}
   \mathbf{v}^l_i = G_v^l(\mathbf{v}^{l-1}_i,\mathbf{m}_{\mathcal{N}(\mathbf{v}^{l-1}_i)})
\end{split}
\label{eq:vertex}
\end{equation}
where $G_v^l$ denotes a differentiable function of the $l_{th}$ GNN layer, which updates all vertex features for producing the graph $\mathcal{G}^{l}$. 



Meanwhile, each edge feature $\mathbf{e}_{i,j}^l$ in the graph $\mathcal{G}^{l}$ can be either kept as the same to its previous status $\mathbf{e}_{i,j}^{l-1}$ in the graph $\mathcal{G}^{l-1}$ \cite{kipf2016semi,hamilton2017inductive} (denoted as the GNN type 1) or updated \cite{gong2019exploiting,jo2021edge,isufi2021edgenets} (denoted as the GNN type 2) during GNNs' propagation. Specifically, each edge feature $\mathbf{e}_{i,j}^l \in \mathcal{G}^{l}$ is computed based on: (i) its previous status $\mathbf{e}^{l-1}_{i,j} \in \mathcal{G}^{l-1}$; and (ii) the corresponding vertex features $\mathbf{v}^{l-1}_i$ and $\mathbf{v}^{l-1}_j$ in $\mathcal{G}^{l-1}$. Mathematically speaking, the $\mathbf{e}^{l}_{i,j}$ can be computed as:
\begin{equation}
\begin{split}
    \mathbf{e}_{i,j}^l = 
    \begin{cases}
    \mathbf{e}^{l-1}_{i,j} &  \text{GNN type 1} \\
    G_e^l(\mathbf{e}^{l-1}_{i,j},g(\mathbf{v}^{l-1}_i, \mathbf{v}^{l-1}_j)) & \text{GNN type 2}
    \end{cases}
\end{split}
\end{equation}
where $G_e^l$ is a differentiable function of the $l_{th}$ GNN layer, which updates edge features to produce the graph $\mathcal{G}^{l}$, and $g$ is a differentiable function that models the relationship between $\mathbf{v}^{l-1}_i$ and $\mathbf{v}^{l-1}_j$. In summary, vertex and edge features' updating are mutually influenced during the propagation of message-passing GNNs. Please refer to Hamilton et al. \cite{hamilton2020graph} and Dwivedi et al. \cite{dwivedi2020benchmarking} for more details.

# Methodology

## Algorithmic Definitions

**Task-specific:** The term refers to features or topology specifically extracted for a particular task. These are produced from the original input data using networks trained with the target labels. In simpler terms, the networks define the features or topology based on the relationship between the input and the label.

**Plug-and-play:** The term refers to the ability to easily integrate the model into a machine learning framework (i.e., Backbone-GNN frameworks in this paper) without requiring significant modifications or customization.

**Edge feature:** The term refers to a vector or a single value attached to a graph edge, which describes the characteristics of the edge based on the relationship between the corresponding vertex pair. In this paper, the term 'multi-dimensional edge feature' represents a vector consisting of multiple attributes to describe the characteristics of the corresponding edge.

## Visualization of GD and TTP Modules

Figure 1 illustrates the GD and TTP modules of the graph-based GRATIS. The GCN in the backbone first produces a set of vertex features from the input graph $\mathcal{D}^{\text{in}}$, which are concatenated as a matrix $X^{\text{GCN}}$. Then, the CNN yields the global contextual representation $X$ from the $X^{\text{GCN}}$. The GD module simply treats the original input graph as the basic graph. After that, the TTP module produces an adjacency probability matrix $\mathcal{\hat{A}}^{\text{prob}}$ from $X$, which is then combined with the basic graph topology to generate the final task-specific adjacency matrix $\mathcal{\hat{A}}$. 

<div align="center">
<img src="figures/submodules_predefined_graphs.png" alt="GCN-CNN Backbone, Graph Definition (GD), and Task-specific Topology Prediction (TTP) modules for processing pre-defined graphs" width="60%" />
</div>

*Figure 1: Illustration of the GCN-CNN Backbone, Graph Definition (GD), and Task-specific Topology Prediction (TTP) modules for processing **pre-defined graphs**.*

Figure 2 illustrates the GD and TTP modules of the non-graph based GRATIS, which can take various types of non-graph data (e.g., image, audio, video, text and time-series). During training, in the GD module, the outputs of the VFE are fed to a standard MLP-based predictor, making the VFE to be intermediately supervised. This allows the trained VFE to provide a set of task-related vertex features, based on which the basic graph is defined. Then, the TTP module jointly trains the VFE with a GCN, allowing a $\hat{\text{VFE}}$ to be further optimized to produce task-specific graph vertex features and a vertex-decided graph topology, which are combined with the basic graph topology to generate the graph $\mathcal{G}^{\text{V}}(\mathcal{\hat{V}},\mathcal{E}^{V})$ that have an optimal task-specific topology.

![GD and TTP modules for **non-graph data**](figures/submodules_non_graphs.png)
*Figure 2: Illustration of the GD and TTP modules for **non-graph data**, which are jointly optimized during training.*

## Framework differentiation analysis

In this section, we show that the three modules (GD, TTP and MEFG) of the proposed framework can be jointly trained with any differentiable backbone and GNN predictor in an end-to-end manner. This allows the weights learned for all modules to be task-specific. 

Firstly, the GD module for processing graph data only contain an identity mapping, while the GD module for processing non-graph data is made up of a set of differentiable FC and GAP layers (i.e., the VFE). Therefore, the GD module for processing both graph and non-graph data are fully differentiable. Let $\mathcal{L}$ as a differentiable objective function that measures the loss of the final prediction generated by the GNN predictor, its gradient w.r.t. the graph $\mathcal{G}^{B}(\mathcal{V}, \mathcal{E})$ generated by the GD module can be computed as:

\begin{equation}
\begin{split}
\frac{\partial \mathcal{L}}{\partial \mathcal{G}^{B}} &= \frac{\partial \mathcal{L}}{\partial p_{\mathcal{\hat{G}}}} \frac{\partial p_{\mathcal{\hat{G}}}}{\partial \mathcal{\hat{G}}} \frac{\partial \mathcal{\hat{G}}}{\partial \mathcal{G}^{B}} \\
& = \frac{\partial \mathcal{L}}{\partial p_{\mathcal{\hat{G}}}} \frac{\partial p_{\mathcal{\hat{G}}}}{\partial \mathcal{\hat{G}}}(\frac{\partial \mathcal{\hat{V}}}{\partial \mathcal{V}}+\frac{\partial \mathcal{\hat{V}}}{\partial X}, \frac{\partial \mathcal{\hat{E}}}{\partial \mathcal{V}}+\frac{\partial \mathcal{\hat{E}}}{\partial X}) \\
& = \frac{\partial \mathcal{L}}{\partial p_{\mathcal{\hat{G}}}} \frac{\partial \text{GNN}(\mathcal{\hat{G}})}{\partial \mathcal{\hat{G}}} (\frac{\partial 
\text{TTP}(X, \mathcal{A}, \mathcal{V})}{\partial \mathcal{V}} + \frac{\partial 
\text{TTP}(X, \mathcal{A}, \mathcal{V})}{\partial X},   \\ 
&\frac{\partial  \text{MEFG}(X, \mathcal{V})}{\partial \mathcal{V}}+\frac{\partial  \text{MEFG}(X, \mathcal{V})}{\partial X})
\end{split}
\label{eq:back-propagation}
\end{equation}

where $p_{\mathcal{\hat{G}}}$ denotes the graph analysis prediction obtained from the differentiable GNN predictor; $\mathcal{\hat{G}}(\mathcal{\hat{V}}, \mathcal{\hat{E}})$ is the final produced task-specific graph representation ($\mathcal{\hat{E}}$ is produced from the MEFG module and $\mathcal{\hat{V}}$ is produced from the TTP module). Here, both GNN and MEFG are made up of fully differentiable layers. In other words, the terms $\frac{\partial \mathcal{L}}{\partial p_{\mathcal{\hat{G}}}}$, $ \frac{\partial \text{GNN}(\mathcal{\hat{G}})}{\partial \mathcal{\hat{G}}}$, $ \frac{\partial  \text{MEFG}(X, \mathcal{V})}{\partial \mathcal{V}}$ and $\frac{\partial  \text{MEFG}(X, \mathcal{V})}{\partial X}$ are differentiable. 

In addition, the generation of $\mathcal{\hat{A}}^{\text{prob}}$ for graph data and $\mathcal{\hat{A}}^{\text{V}}$ for non-graph data are fully decided by global contextual representation $X$ and task-specific vertex features $\mathcal{\hat{V}}$, both of which are produced via fully differentiable operations, i.e., $X$ is generated by a CNN/GCN backbone, and $\mathcal{\hat{V}}$ is produced by either identity mapping for pre-defined graphs or the VFE for non-graph data. As a result, the gradients for updating $\mathcal{A}$ to $\mathcal{\hat{A}}^{\text{prob}}$ or $\mathcal{\hat{A}}^{\text{V}}$ can be formulated as:

\begin{equation}
\begin{split}
\frac{\partial \mathcal{L}}{\partial \mathcal{A}} &= \frac{\partial \mathcal{L}}{\partial \mathcal{V}}+\frac{\partial \mathcal{L}}{\partial X} \\
&= \frac{\partial \mathcal{L}}{\partial p_{\mathcal{\hat{G}}}} 
\frac{\partial p_{\mathcal{\hat{G}}}}{\partial \mathcal{\hat{G}}} (\frac{\partial \mathcal{\hat{G}}}{\partial \mathcal{V}} + \frac{\partial \mathcal{\hat{G}}}{\partial X} )\\
&= \frac{\partial \mathcal{L}}{\partial p_{\mathcal{\hat{G}}}} 
\frac{\partial p_{\mathcal{\hat{G}}}}{\partial \mathcal{\hat{G}}} (\frac{\partial \mathcal{\hat{V}}}{\partial \mathcal{V}} + \frac{\partial \mathcal{\hat{E}}}{\partial \mathcal{V}} + \frac{\partial \mathcal{\hat{V}}}{\partial X} + \frac{\partial \mathcal{\hat{E}}}{\partial X})
\end{split}
\label{eq:back-propagation_A}
\end{equation}

As analyzed in Eqa. \ref{eq:back-propagation}, all terms in Eqa. \ref{eq:back-propagation_A} are differentiable, which means that the back-propagated gradients would enforce the adjacency matrices $\mathcal{\hat{A}}^{\text{prob}}$ or $\mathcal{\hat{A}}^{\text{V}}$ to be also learned in an end-to-end manner. Although the process that updates the adjacency matrix from $\mathcal{A}$ to $\mathcal{\hat{A}}$ (Eqa. (19) of the main document) in the TTP module is not fully differentiable, to the best of our knowledge, existing GNNs do not forward the adjacency matrix within the network, i.e., the Eqa. \ref{eq:back-propagation} does not need to compute $\frac{\partial 
\text{TTP}(X, \mathcal{A}, \mathcal{V})}{\partial A}$. Consequently, the proposed approach is a fully differentiable framework.

## Detailed results of AU recognition task (non-graph vertex classification)

We report the detailed F1 and AUC results of the AU recognition task (i.e., non-graph-based vertex classification), which are achieved by our approach with different settings on BP4D and DISFA datasets. These are provided in Table \ref{ex:tab_BP4D_sota}, Table \ref{ex:tab_AUC_BP4D}, Table \ref{ex:tab_DISFA_sota}, and Table \ref{ex:tab_AUC_DISFA}. We also compare them in Figure 3 and Figure 4.

![f1_auc_BP4D](figures/f1_auc_BP4D.png)
*Figure 3: F1 scores and AUC results (\%) achieved for 12 AUs' recognition on BP4D dataset, where three competitors (SRERL, UGN-B and HMP-PS) are also graph-based methods.*

![f1_auc_BP4D](figures/f1_auc_DISFA.png)
*Figure 4: F1 scores and AUC results (\%) achieved for 8 AUs' recognition on DISFA dataset, where three competitors (SRERL, UGN-B and HMP-PS) are also graph-based methods.*
